In [1]:
import snowflake.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# connection parameters
account = 'rv79164.mexico-central.azure'
user = 'anguiegarcia'
password = 'AnguieCobre_2025'
database = 'my_database'
schema = 'my_schema'
warehouse = 'my_warehouse'
role = 'accountadmin'
threads = 1

# Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    database=database,
    schema=schema,
    warehouse=warehouse,
    role=role
)

In [3]:

sql_query = "SELECT * FROM IN_HUBSPOT_LEADS_ENRICHED;"
df = pd.read_sql(sql_query, conn)
print(f"shape: {df.shape}")
df.head()

C:\Users\BETAN\AppData\Local\Temp\ipykernel_28844\99317036.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


shape: (8000, 12)


,MQL_ID,FIRST_CONTACT_DATE,LEAD_SOURCE,LANDING_PAGE_ID,SELLER_ID,WON_DATE,DAYS_TO_CONVERSION,IS_WON,BUSINESS_SEGMENT,LEAD_TYPE,BUSINESS_TYPE,LEAD_BEHAVIOUR_PROFILE
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,social,88740e65d5d6b056e0cda098e1ea6313,None,NaT,NaN,0,None,None,None,None
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,paid_search,007f9098284a86ee80ddeb25d53e0af8,None,NaT,NaN,0,None,None,None,None
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,organic_search,a7982125ff7aa3b2054c6e44f9d28522,None,NaT,NaN,0,None,None,None,None
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,email,d45d558f0daeecf3cccdffe3c59684aa,None,NaT,NaN,0,None,None,None,None
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,organic_search,b48ec5f3b04e9068441002a19df93c6c,2c43fb513632d29b3b58df74816f1b06,2018-02-26 19:58:54,5.0,1,pet,online_medium,reseller,cat


In [85]:
df['LEAD_SOURCE'].value_counts()

LEAD_SOURCE
organic_search       2296
paid_search          1586
social               1350
unknown              1159
direct_traffic        499
email                 493
referral              284
other                 150
display               118
other_publicities      65
Name: count, dtype: int64

## 📈 Logistic Regression Attribution

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [7]:
# Create a list of behaviors to check for (cat, eagle, wolf, shark)
behaviors = ['LEAD_cat', 'LEAD_eagle', 'LEAD_wolf', 'LEAD_shark']

# Create a column for each behavior (cat, eagle, wolf, shark)
for behavior in behaviors:
    df[behavior] = df['LEAD_BEHAVIOUR_PROFILE'].apply(lambda x: 1 if isinstance(x, str) and behavior in x else 0)

# Get dummies
dumis = pd.get_dummies(df[['LEAD_SOURCE', 'LEAD_TYPE']], drop_first=True)

df = pd.concat([df, dumis], axis=1)
# Columnas para X (features) y y (target)
categorical_columns = ['LEAD_cat', 'LEAD_eagle','LEAD_wolf', 'LEAD_shark',
        'LEAD_SOURCE_email', 'LEAD_SOURCE_organic_search', 'LEAD_SOURCE_other',
       'LEAD_SOURCE_other_publicities', 'LEAD_SOURCE_paid_search',
       'LEAD_SOURCE_referral', 'LEAD_SOURCE_social', 'LEAD_SOURCE_unknown',
       'LEAD_TYPE_offline', 'LEAD_TYPE_online_beginner',
       'LEAD_TYPE_online_big', 'LEAD_TYPE_online_medium',
       'LEAD_TYPE_online_small', 'LEAD_TYPE_online_top', 'LEAD_TYPE_other']  

X = df[categorical_columns]

y = df['IS_WON'] 

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict the conversions on the test set
y_pred = model.predict(X_test)

# Evaluate the performance of the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# Importancia de features
importances = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)
print(importances)

Accuracy: 0.9875
Confusion Matrix:
 [[2162    0]
 [  30  208]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      2162
           1       1.00      0.87      0.93       238

    accuracy                           0.99      2400
   macro avg       0.99      0.94      0.96      2400
weighted avg       0.99      0.99      0.99      2400

LEAD_TYPE_online_medium          7.086497
LEAD_TYPE_online_big             6.232904
LEAD_TYPE_offline                6.062996
LEAD_TYPE_online_small           5.959777
LEAD_TYPE_online_beginner        5.373419
LEAD_TYPE_online_top             4.256789
LEAD_TYPE_other                  2.024004
LEAD_SOURCE_unknown              0.672588
LEAD_SOURCE_paid_search          0.604277
LEAD_SOURCE_organic_search       0.387046
LEAD_SOURCE_referral             0.138011
LEAD_wolf                        0.000000
LEAD_eagle                       0.000000
LEAD_cat                         0.00000